## Data preparation

In [45]:
import pandas as pd
import numpy as np

In [57]:
data = pd.read_csv("C:/Users/rafal/Desktop/DS/Merceri price/data_selected.csv")

In [58]:
data = data.sample(n = 50000)

In [48]:
data.head()

,Unnamed: 0,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
59009,833571,833571,Bobbins widger,2,Handmade/Patterns/Sewing,NaN,14.0,0,A bobbins widger with charger
14951,1306464,1306464,Fashion Nova Pants,2,"Women/Jeans/Slim, Skinny",Fashion Nova,15.0,1,Like new..Never worn just tried on..Size XL fa...
66940,937940,937940,Slip on shoes,2,Women/Shoes/Loafers & Slip-Ons,NaN,14.0,1,Clarks slip on blue slightly textured shoes. M...
60228,1443078,1443078,"Samsung Galaxy tab A 8"" case & protector",1,"Electronics/Cell Phones & Accessories/Cases, C...",Samsung,20.0,1,"Samsung Galaxy tab A 8.0"" case & 2 tempered gl..."
96029,1395233,1395233,2 piece,2,Women/Suits & Blazers/Pant Suit,Fashion Nova,10.0,0,"Velvet , nice summer outfit"


In [59]:
data.drop(columns = ['Unnamed: 0', 'train_id'], axis = 1, inplace = True)

### Categories

In [51]:
def StringToColumns(data, column_name, sep, split_limit, fill_na = "No_category"):
    categories = data[column_name].str.split(sep, split_limit, expand = True)
    categories = categories.drop(columns = 3, axis = 1)
    categories = categories.fillna(value = fill_na)
    data_cats = pd.concat([data, categories], axis = 1)
    data_features = data_cats.drop(columns = [column_name], axis = 1)
    return data_features

def WordsToColumns(data, max_columns, column_number):
    import re
    import nltk
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    from sklearn.feature_extraction.text import CountVectorizer
    corpus = []
    for i in range(0, len(data)):
        name = re.sub('[^a-zA-Z]', ' ', data.iloc[i, column_number])
        name = name.lower()
        name = name.split()
        ps = PorterStemmer()
        name = [ps.stem(word) for word in name if not word in set(stopwords.words('english'))]
        name = ' '.join(name)
        corpus.append(name)    
    # Creating the Bag of Words model
    cv = CountVectorizer(max_features = max_columns)
    name_features = cv.fit_transform(corpus)
    df_name_features = pd.DataFrame(name_features.toarray())
    return df_name_features

def SelectedLevelsToColumns_count(data, target, categorical, percentage=0.001, na_fill = 'no_brand_name'):
    from sklearn.preprocessing import OneHotEncoder
    pivot_brands = pd.pivot_table(data, values = target, index = categorical, aggfunc = ['count'])
    pivot_brands.columns = pivot_brands.columns.get_level_values(0)
    pv_sorted = pivot_brands.sort_values(by = ['count'], ascending = False)
    selected_brands = pv_sorted[pv_sorted['count']>percentage*len(data)]
    brand_list = selected_brands.index
    brand_list = np.array(brand_list)
    ohe = OneHotEncoder()
    cat_data = pd.DataFrame(data[categorical])
    cat_data.fillna(value = na_fill, inplace = True)
    data['new_categorical'] = np.where(data[categorical].isin(brand_list), data[categorical], 'other')
    categories = ohe.fit_transform(pd.DataFrame(data['new_categorical']))
    data.drop(columns=['new_categorical', categorical], axis = 1, inplace = True)
    data.reset_index(drop = True, inplace = True)
    data_final = pd.concat([data, pd.DataFrame(categories.toarray())], axis = 1)
    #Return also the brand_list for preprocesssing the test set
    return data_final

In [60]:
data = StringToColumns(data, 'category_name', '/', 3)

In [61]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
data_categories = ohe.fit_transform(data[[0, 1, 2]])

In [62]:
data_categories = pd.DataFrame(data_categories.toarray())

In [66]:
data.reset_index(inplace = True)

In [67]:
data = pd.concat([data, data_categories], axis = 1)

In [72]:
data.drop(columns = [0,1,2], axis = 1, inplace=True)

KeyError: '[0 1 2] not found in axis'

In [73]:
data.drop(columns = 'index', inplace = True)

In [74]:
data.head()

,name,item_condition_id,brand_name,price,shipping,item_description,3,4,5,6,...,732,733,734,735,736,737,738,739,740,741
0,NWT Simply Southern Shirt,1,NaN,16.0,1,New with tags size large,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,UFC 2,1,NaN,39.0,0,New,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Olay orchid set,1,NaN,50.0,0,No free ship! Price is firm due to merc's 10% ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Shopkins Season3 Cupcake Collection,2,Moose Toys,10.0,0,No description yet,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,(M) Victoria's secret,3,Victoria's Secret,12.0,1,"VS pj bottoms colors are pink, white, and blac...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Name

In [75]:
data['name'].fillna(value = 'blank', inplace = True)

In [76]:
data_name = WordsToColumns(data, 1000, 0)

In [29]:
data_name.to_csv("C:/Users/rafal/Desktop/DS/Merceri price/data_name_preprocessed.csv")

In [10]:
data_name = pd.read_csv("C:/Users/rafal/Desktop/DS/Merceri price/data_name_preprocessed.csv")

In [77]:
data_with_name = pd.concat([data, data_name], axis = 1)

In [79]:
data_with_name.head()

(50000, 1745)

In [80]:
data_with_name.drop(columns= 'name', axis = 1, inplace=True)

### Brand name

In [81]:
data_with_brand_name = SelectedLevelsToColumns_count(data_with_name, 'price', 'brand_name')

In [82]:
data_with_brand_name.head()

,item_condition_id,price,shipping,item_description,3,4,5,6,7,8,...,95,96,97,98,99,100,101,102,103,104
0,1,16.0,1,New with tags size large,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,39.0,0,New,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,50.0,0,No free ship! Price is firm due to merc's 10% ...,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2,10.0,0,No description yet,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,3,12.0,1,"VS pj bottoms colors are pink, white, and blac...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Description

In [37]:
data.head()

,name,item_condition_id,brand_name,price,shipping,item_description,3,4,5,6,...,726,727,728,729,730,731,732,733,734,735
0,Chanel Coco Mademoiselle,1.0,Chanel,81.0,0.0,It's brand new and 100% authentic. This is a g...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,blank,NaN,NaN,NaN,NaN,,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Nwt Northface Reversible Jacket xs 6,1.0,The North Face,56.0,0.0,Girls extra small xs 6 Northface jacket Super ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,blank,NaN,NaN,NaN,NaN,,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ONE PIECE bathing suit,2.0,NaN,17.0,0.0,This is just for the one piece! Size small/medium,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
data['item_description'].fillna(value = "", inplace = True)

In [84]:
data_description = WordsToColumns(data,2500,5)

In [39]:
data_description.to_csv("C:/Users/rafal/Desktop/DS/Merceri price/data_description_preprocessed.csv")

In [14]:
data_description = pd.read_csv("C:/Users/rafal/Desktop/DS/Merceri price/data_description_preprocessed.csv")

In [85]:
data_final = pd.concat([data_with_brand_name, data_description], axis = 1)

In [87]:
data_final.drop(columns = 'item_description', inplace = True)

In [88]:
del data, data_categories, data_description, data_name, data_with_brand_name, data_with_name

In [89]:
data_final.to_csv("C:/Users/rafal/Desktop/DS/Merceri price/data_final.csv")

In [20]:
data_final.head()

,item_condition_id,price,shipping,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,1,81.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,3,13.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1,56.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2,8.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2,17.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
